In [16]:
import findspark
findspark.init()

from pyspark import  SparkContext
from pyspark.sql import SparkSession

spark_session = SparkSession\
    .builder\
    .master("spark://192.168.2.243:7077") \
    .appName("test")\
    .config("spark.executor.cores", 2)\
    .getOrCreate()

spark_context = spark_session.sparkContext

In [17]:
df1 = spark_session.read\
.option("header","true")\
.option("delimiter", "\t")\
.csv("hdfs://team-4-master:9000/1000genomes/All.chr22.E1129_UNION_GL.20110521.snp.exome.genotypes_processed.vcf")

In [18]:
# Filter out others and only keep the SNP alleles data
df2 = df1.select("REF","ALT")

In [19]:
# Combine the two column so that now we have a column showing the mutation of the SNP
# e.g. mutated from a G to an A

df2.createOrReplaceTempView("df2")
df3 = spark_session.sql("SELECT CONCAT(REF, 'to',  ALT) as Mutation FROM df2")

In [29]:
# WE now can know which mutation is the most common one in this chromosome.
df4 = df3.select("Mutation").count().sort("count", ascending=False).show()

AttributeError: 'int' object has no attribute 'sort'

In [5]:
# WE now can know which mutation is the most common one in this chromosome.
df4 = df3.select("Mutation").groupBy("Mutation").count().sort("count", ascending=False).show()

+--------+------+
|Mutation| count|
+--------+------+
|    GtoA|114818|
|    CtoT|113628|
|    TtoC| 55770|
|    AtoG| 55603|
|    CtoG| 21970|
|    GtoC| 21568|
|    GtoT| 20552|
|    CtoA| 20481|
|    AtoC| 13759|
|    TtoG| 13514|
|    AtoT| 11997|
|    TtoA| 11711|
+--------+------+



In [15]:
spark_context.stop()